In [1]:
from surprise import accuracy, BaselineOnly, Dataset, SVD, Reader
from surprise.model_selection import cross_validate, KFold

# path to dataset file
file_path = 'ml-1m/ratings_timesorted.dat'

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating timestamp', sep='::', rating_scale=(-2, 2))

trainset = Dataset.load_from_file(file_path, reader).build_full_trainset()
#trainset = Dataset.load_builtin('ml-1m').build_full_trainset()
testset = trainset.build_testset()

(-2, 2)

In [27]:
algo = SVD(n_factors=8, biased=False, init_std_dev=0.1, n_epochs=100, reg_all=0.005, lr_all=0.005)
algo.fit(trainset)
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)  # ~ 0.68 (which is low)

RMSE: 1.2594


1.2594175234241636

In [ ]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

# We can also do this during a cross-validation procedure!
print('CV procedure:')

kf = KFold(n_splits=3)
for i, (trainset_cv, testset_cv) in enumerate(kf.split(data)):
    print('fold number', i + 1)
    algo.fit(trainset_cv)

    print('On testset,', end='  ')
    predictions = algo.test(testset_cv)
    accuracy.rmse(predictions, verbose=True)

    print('On trainset,', end=' ')
    predictions = algo.test(trainset_cv.build_testset())
    accuracy.rmse(predictions, verbose=True)